In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split


In [26]:
data = pd.read_csv("../CSV/big_files/ais_train_processed.csv", sep="|")
data.drop(columns=["portId"], inplace=True)

X = data.copy().drop(columns=["latitude", "longitude"])
y = data.copy()[["latitude", "longitude"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

""" X_train = pd.get_dummies(X_train, columns=["vesselId"], drop_first=True)
X_test = pd.get_dummies(X_test, columns=["vesselId"], drop_first=True)


#More data BS
for col in X_train:
    if col not in X_test:
        X_test[col] = False

X_train = X_train.reindex(sorted(X_train.columns), axis=1)
X_test = X_test.reindex(sorted(X_test.columns), axis=1) """

X_train.reset_index(drop=False, inplace=True)
X_test.reset_index(drop=False, inplace=True)

X_train = X_train.set_index(["index", "time"])
X_test = X_test.set_index(["index", "time"])



def evaluate(model: str, y_pred):

    n = len(y_test)
    p = X_test.shape[1]

    mse_latitude = mean_squared_error(y_test, y_pred)
    mse_longitude = mean_squared_error(y_test, y_pred)
    r2_latitude = r2_score(y_test, y_pred)
    r2_latitude_adj = 1 - (1 - r2_latitude) * ((n - 1) / (n - p - 1))
    r2_longitude = r2_score(y_test, y_pred)
    r2_longitude_adj = 1 - (1 - r2_longitude) * ((n - 1) / (n - p - 1))


    print(f"---- {model} Metrics ----")
    print(f"Mean Squared Error (Latitude): {mse_latitude:.4f}")
    print(f"R-squared (Latitude): {r2_latitude:.4f}, Adjusted R-squared (Latitude): {r2_latitude_adj: .4f}")
    print(f"Mean Squared Error (Longitude): {mse_longitude:.4f}")
    print(f"R-squared (Longitude): {r2_longitude:.4f}, Adjusted R-squared (Latitude): {r2_longitude_adj: .4f}")

X_train.head()

,,cog,sog,rot,heading,navstat,etaRaw,vesselId,drift,at_port,time_at_sea
time,index,,,,,,,,,,
1704067225,0,284.0,0.7,0,88,0,1704841200,61e9f3a8b937134a3c4bfdf7,196.0,False,0.0
1704067236,1,109.6,0.0,-6,347,1,1735502400,61e9f3d4b937134a3c4bff1f,-237.4,False,0.0
1704067305,2,111.0,11.0,0,112,0,1704186000,61e9f436b937134a3c4c0131,-1.0,False,0.0
1704067391,3,96.4,0.0,0,142,1,1735675200,61e9f3b4b937134a3c4bfe77,-45.6,False,0.0
1704067431,4,214.0,19.7,0,215,0,1706184000,61e9f41bb937134a3c4c0087,-1.0,False,0.0


In [24]:
# --- Time Series KNN ---
from sktime.regression.distance_based import KNeighborsTimeSeriesRegressor

start = time.time()
KNNTS = KNeighborsTimeSeriesRegressor(n_neighbors=3, distance="euclidean")
KNNTS.fit(X_train, y_train)
print(time.time()-start)

knnts_pred = KNNTS.predict(X_test)

evaluate("KNNTS", knnts_pred)

TypeError: X must be in an sktime compatible format. Allowed scitypes for regressors are Panel mtypes, for instance a pandas.DataFrame with MultiIndex and last(-1) level an sktime compatible time index. Allowed compatible mtype format specifications are: ['nested_univ', 'numpy3D', 'numpyflat', 'pd-multiindex', 'pd-wide', 'pd-long', 'df-list', 'gluonts_ListDataset_panel', 'gluonts_PandasDataset_panel', 'polars_panel'] . See the data format tutorial examples/AA_datatypes_and_datasets.ipynb. If you think the data is already in an sktime supported input format, run sktime.datatypes.check_raise(data, mtype) to diagnose the error, where mtype is the string of the type specification you want. Error message for checked mtypes, in format [mtype: message], as follows: [df-list: obj must be list of pd.DataFrame, found <class 'pandas.core.frame.DataFrame'>]  [numpy3D: obj must be a numpy.ndarray, found <class 'pandas.core.frame.DataFrame'>]  [pd-multiindex: <class 'pandas.core.indexes.multi.MultiI